In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *
from executor import *
from dfs import *
from dfs_viewer import *
from main import reset_grid

In [2]:
grid = reset_grid()
grid.show()
dfs = DFS(grid, 7)

new score 35 != old score 37 at MOVE (7, 3) to (6, 2)_BOULDER SHELL at (4, 2)_MOVE (3, 2) to (1, 2)_CLUSTER SHELL at (1, 5)_MOVE (5, 3) to (3, 2)_HEAL +1_
[MOVE (7, 3) to (5, 4), BOULDER SHELL at (2, 4), MOVE (5, 3) to (6, 2), HEAL +1, MOVE (3, 2) to (1, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (3, 2), HEAL +1, MOVE (3, 2) to (3, 2), CLUSTER SHELL at (6, 2), HEAL +1, MOVE (3, 2) to (3, 5), CLUSTER SHELL at (3, 2), CLUSTER SHELL at (7, 5), HEAL +1, MOVE (3, 2) to (3, 3), HEAL +1, MOVE (3, 2) to (2, 1), CLUSTER SHELL at (7, 1), CLUSTER SHELL at (4, 1), CLUSTER SHELL at (5, 1), HEAL +1, MOVE (3, 2) to (2, 0), CLUSTER SHELL at (6, 0), HEAL +1, MOVE (3, 2) to (2, 3), CLUSTER SHELL at (4, 3), HEAL +1, REGULAR SHELL at (6, 6), MOVE (3, 2) to (1, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (3, 2), CLUSTER SHELL at (5, 2), HEAL +1, MOVE (3, 2) to (3, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (5, 2), HEAL +1, MOVE (3, 2) to (3, 5), CLUSTER SHELL at (3, 2), HEAL +1, MOVE (3, 2) to (3, 

new score -4 != old score -2 at MOVE (7, 3) to (6, 2)_BOULDER SHELL at (4, 2)_MOVE (3, 2) to (1, 2)_CLUSTER SHELL at (1, 5)_MOVEMOVE (5, 3) to (5, 4)_
[MOVE (7, 3) to (5, 4), BOULDER SHELL at (2, 4), MOVE (5, 3) to (6, 2), HEAL +1, MOVE (3, 2) to (1, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (3, 2), HEAL +1, MOVE (3, 2) to (3, 2), CLUSTER SHELL at (6, 2), HEAL +1, MOVE (3, 2) to (3, 5), CLUSTER SHELL at (3, 2), CLUSTER SHELL at (7, 5), HEAL +1, MOVE (3, 2) to (3, 3), HEAL +1, MOVE (3, 2) to (2, 1), CLUSTER SHELL at (7, 1), CLUSTER SHELL at (4, 1), CLUSTER SHELL at (5, 1), HEAL +1, MOVE (3, 2) to (2, 0), CLUSTER SHELL at (6, 0), HEAL +1, MOVE (3, 2) to (2, 3), CLUSTER SHELL at (4, 3), HEAL +1, REGULAR SHELL at (6, 6), MOVE (3, 2) to (1, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (3, 2), CLUSTER SHELL at (5, 2), HEAL +1, MOVE (3, 2) to (3, 2), CLUSTER SHELL at (6, 2), CLUSTER SHELL at (5, 2), HEAL +1, MOVE (3, 2) to (3, 5), CLUSTER SHELL at (3, 2), HEAL +1, MOVE (3, 2) to (3, 3), 

In [3]:
grid = reset_grid()
DFSViewer(grid, dfs)

54


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,Siege Mech ♡♡♡♡,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Artillery Mech ♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,🟢,Alpha Firefly ♡♡♡♡,🟢,🟢,🌲
6,🗻,🟢,Boulder Mech ♡♡♡♡♡,🔥,🔥,Firefly ♡♡♡,🟢,🟢
7,🗻,🗻,🟢,🌲,🟢,Boulder ♡,🟢,🗻


MOVE (7, 3) to (6, 2)
BOULDER SHELL at (4, 2)
MOVE (3, 2) to (1, 2)
CLUSTER SHELL at (6, 2)
MOVE (5, 3) to (3, 2)
REGULAR SHELL at (3, 4)

53


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,Siege Mech ♡♡♡♡,Boulder 2 ♡,🗻,🗻,💦
3,🟢,🏘️ ϟ,🟢,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡,🏘️ ϟ,🏘️ ϟϟ,🟢
5,🟢,Scarab ♡♡,🔥,🟢,Boulder Mech ♡♡♡♡♡,🟢,🟢,🌲
6,🗻,🟢,⬆️,🔥,🔥,Firefly ♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Artillery Mech ♡,🟢,Boulder ♡,🟢,🗻


MOVE (7, 3) to (5, 4)
BOULDER SHELL at (2, 4)
MOVE (3, 2) to (2, 3)
MOVE (5, 3) to (7, 3)
CLUSTER SHELL at (4, 3)
REGULAR SHELL at (7, 1)

53


...

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,Siege Mech ♡♡♡♡,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,🟢,🟢,Boulder 2 ♡,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡,🏘️ ϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,🟢,Boulder Mech ♡♡♡♡♡,🟢,🟢,🌲
6,🗻,Scarab ♡♡,Artillery Mech ♡♡,🔥,🔥,Firefly ♡♡♡,🟢,🟢
7,🗻,🗻,🟢,🌲,🟢,Boulder ♡,🟢,🗻


MOVE (7, 3) to (5, 4)
BOULDER SHELL at (3, 4)
MOVE (5, 3) to (6, 2)
HEAL +1
MOVE (3, 2) to (2, 3)
CLUSTER SHELL at (4, 3)

49


,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,Siege Mech ♡♡♡♡,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,🟢,🟢,Boulder 2 ♡,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡,🟢,Alpha Firefly ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,🟢,Boulder Mech ♡♡♡♡♡,🟢,🟢,🌲
6,🗻,Artillery Mech ♡,Scarab ♡,🔥,Firefly ♡♡♡♡,🟢,🟢,🟢
7,🗻,🗻,🟢,🌲,🟢,Boulder ♡,🟢,🗻


MOVE (7, 3) to (5, 4)
BOULDER SHELL at (3, 4)
MOVE (3, 2) to (2, 0)
CLUSTER SHELL at (6, 0)
MOVE (5, 3) to (6, 1)
REGULAR SHELL at (6, 6)

49


AssertionError: BOULDER SHELL at (2, 4) is not in {'MOVE (3, 2) to (1, 2)': MOVE (3, 2) to (1, 2), 'MOVE (3, 2) to (3, 2)': MOVE (3, 2) to (3, 2), 'MOVE (3, 2) to (3, 5)': MOVE (3, 2) to (3, 5), 'MOVE (3, 2) to (3, 3)': MOVE (3, 2) to (3, 3), 'MOVE (3, 2) to (0, 2)': MOVE (3, 2) to (0, 2), 'MOVE (3, 2) to (2, 1)': MOVE (3, 2) to (2, 1), 'MOVE (3, 2) to (2, 0)': MOVE (3, 2) to (2, 0), 'MOVE (3, 2) to (2, 3)': MOVE (3, 2) to (2, 3), 'MOVE (3, 2) to (4, 3)': MOVE (3, 2) to (4, 3), 'MOVE (3, 2) to (2, 2)': MOVE (3, 2) to (2, 2), 'MOVE (3, 2) to (3, 4)': MOVE (3, 2) to (3, 4), 'MOVE (3, 2) to (2, 4)': MOVE (3, 2) to (2, 4), 'MOVE (3, 2) to (5, 3)': MOVE (3, 2) to (5, 3), 'HEAL +1': HEAL +1, 'REGULAR SHELL at (0, 2)': REGULAR SHELL at (0, 2), 'REGULAR SHELL at (1, 2)': REGULAR SHELL at (1, 2), 'REGULAR SHELL at (2, 2)': REGULAR SHELL at (2, 2), 'REGULAR SHELL at (3, 2)': REGULAR SHELL at (3, 2), 'REGULAR SHELL at (4, 2)': REGULAR SHELL at (4, 2), 'REGULAR SHELL at (6, 0)': REGULAR SHELL at (

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
DFS.rate(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
stop

In [ ]:
grid = reset_grid()

In [ ]:
grid.show()

In [ ]:
DFS.rate_dict(grid)

In [ ]:
dfs.show()

In [ ]:
dfs.explored

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
sm.ge

In [ ]:
choice.command.__dict__.copy()

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
ex = Executor()

In [ ]:
ex.execute(siege_mech.gen_actions()[2])

In [ ]:
grid.show()

In [ ]:
ex.execute(siege_mech.gen_actions()[4])

In [ ]:
grid.show()

In [ ]:
ex

In [ ]:
ex.undo()

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
ex = Executor()

score = DFS.rate(grid)

DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[2][0])
score = DFS.rate(grid)

show = grid.show()

dic = DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
score = DFS.rate(grid)

frontier.sort(key=lambda x:x[1], reverse=True)

score = DFS.rate(grid)

ex.execute(frontier[0][0])
score = DFS.rate(grid)

dic = DFS.rate_dict(grid, True)

show = grid.show()

In [ ]:
print(score)

In [ ]:
ex.undo()
score = DFS.rate(grid)

show = grid.show()

ex.execute(frontier[1][0])
score = DFS.rate(grid)

grid.show()

ex.undo()
score = DFS.rate(grid)

grid.show()

ex.execute(frontier[2][0])
score = DFS.rate(grid)

grid.show()

grid.get_tile((3,6)).visitor

grid.get_tile((5,5)).visitor

grid.get_tile((4,6)).type_object.health

grid.get_tile((4,5)).type_object.health

DFS.rate_dict(grid, True)

grid.show()

In [ ]:
print(score)

new score 42 != old score 37 at 
MOVE (7, 3) to (5, 5)
MOVE (3, 2) to (3, 4)
CLUSTER SHELL at (3, 2)
BOULDER SHELL at (5, 1)
MOVE (5, 3) to (3, 3)
HEAL +1

[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, 'UNDO', REGULAR SHELL at (7, 1), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 2), HEAL +1, 'UNDO', REGULAR SHELL at (6, 2), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 3), REGULAR SHELL at (3, 6), 'UNDO', REGULAR SHELL at (3, 7), 'UNDO', HEAL +1]

In [ ]:
stop

In [ ]:
stop

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
grid.show()

In [ ]:
frontier = DFS.gen_rated_frontier(grid)

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    

In [ ]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

In [ ]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))